# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [1]:
#pip freeze

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('../WeatherPy/weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Souillac,-20.5167,57.5167,294.46,81,13,4.78,MU,1631718069
1,1,Rikitea,-23.1203,-134.9692,296.25,73,64,7.59,PF,1631718277
2,2,Kaeo,-35.1000,173.7833,288.89,97,100,3.03,NZ,1631718277
3,3,Busselton,-33.6500,115.3333,284.31,83,100,9.22,AU,1631718277
4,4,Lebu,-37.6167,-73.6500,284.33,68,1,2.07,CL,1631718278


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
weather_df.dropna(inplace = True) 
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Souillac,-20.5167,57.5167,294.46,81,13,4.78,MU,1631718069
1,1,Rikitea,-23.1203,-134.9692,296.25,73,64,7.59,PF,1631718277
2,2,Kaeo,-35.1000,173.7833,288.89,97,100,3.03,NZ,1631718277
3,3,Busselton,-33.6500,115.3333,284.31,83,100,9.22,AU,1631718277
4,4,Lebu,-37.6167,-73.6500,284.33,68,1,2.07,CL,1631718278


In [6]:
#Remove Unnamed:0 column
weather_df.drop(weather_df.columns[weather_df.columns.str.contains('Unnamed: 0',case = False)],axis = 1, inplace = True)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Souillac,-20.5167,57.5167,294.46,81,13,4.78,MU,1631718069
1,Rikitea,-23.1203,-134.9692,296.25,73,64,7.59,PF,1631718277
2,Kaeo,-35.1000,173.7833,288.89,97,100,3.03,NZ,1631718277
3,Busselton,-33.6500,115.3333,284.31,83,100,9.22,AU,1631718277
4,Lebu,-37.6167,-73.6500,284.33,68,1,2.07,CL,1631718278


In [7]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df['Humidity']

#Create map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
weather_df['Humidity'].describe()
weather_df['Wind Speed'].describe()
weather_df['Cloudiness'].describe()

count    585.000000
mean      53.991453
std       39.775719
min        0.000000
25%        8.000000
50%       64.000000
75%       94.000000
max      100.000000
Name: Cloudiness, dtype: float64

In [27]:
#Create ideal weather conditions
ideal_df = weather_df.copy()
ideal_df = weather_df[weather_df['Humidity'] <= 75]
ideal_df = weather_df[weather_df['Wind Speed'] <= 8]
ideal_df = weather_df[weather_df['Max Temp'] <= 90]
ideal_df = weather_df[weather_df['Max Temp'] >= 70]
ideal_df = weather_df[weather_df['Cloudiness'] == 0]

ideal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 6 to 571
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        69 non-null     object 
 1   Lat         69 non-null     float64
 2   Lng         69 non-null     float64
 3   Max Temp    69 non-null     float64
 4   Humidity    69 non-null     int64  
 5   Cloudiness  69 non-null     int64  
 6   Wind Speed  69 non-null     float64
 7   Country     69 non-null     object 
 8   Date        69 non-null     int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 5.4+ KB


In [30]:
#Remove null and reset index 
ideal_df = ideal_df.dropna()
ideal_df = ideal_df.reset_index(drop=True)

In [33]:
ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Morón,-34.6534,-58.6198,291.49,39,0,2.24,AR,1631718278
1,Dubai,25.2582,55.3047,309.31,56,0,3.60,AE,1631718232
2,Brønnøysund,65.4625,12.1997,288.15,55,0,4.12,NO,1631718281
3,Saint-Pierre,-21.3393,55.4781,295.97,70,0,4.63,RE,1631718282
4,Dāmghān,36.1683,54.3480,302.23,10,0,2.05,IR,1631718284


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = ideal_df.copy()
hotel_df['Hotel Name'] = np.nan
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Morón,-34.6534,-58.6198,291.49,39,0,2.24,AR,1631718278,NaN
1,Dubai,25.2582,55.3047,309.31,56,0,3.60,AE,1631718232,NaN
2,Brønnøysund,65.4625,12.1997,288.15,55,0,4.12,NO,1631718281,NaN
3,Saint-Pierre,-21.3393,55.4781,295.97,70,0,4.63,RE,1631718282,NaN
4,Dāmghān,36.1683,54.3480,302.23,10,0,2.05,IR,1631718284,NaN


In [49]:
#Set parameters to search for hotels with 5000 meters.
hotels = []
x=0

#Create a for loop to search through cities by coordinates
for cities in hotel_df['City']:
    coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    radius = 5000
    type = 'lodging'
    params = {
    'location' : coordinates,
    "radius": radius,
    "types": type,
    "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#try except in case there is not a hotel available 
    try:
        response = requests.get(base_url, params=params)
        results = response.json()
        hotels.append(results['results'][0]['name'])
    except IndexError:
        hotels.append(np.nan)

#Add hotels to hotels_df
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Morón,-34.6534,-58.6198,291.49,39,0,2.24,AR,1631718278,Hotel Morón
1,Dubai,25.2582,55.3047,309.31,56,0,3.60,AE,1631718232,"Radisson Blu Hotel, Dubai Deira Creek"
2,Brønnøysund,65.4625,12.1997,288.15,55,0,4.12,NO,1631718281,Corner Hotel
3,Saint-Pierre,-21.3393,55.4781,295.97,70,0,4.63,RE,1631718282,Lindsey Hôtel
4,Dāmghān,36.1683,54.3480,302.23,10,0,2.05,IR,1631718284,خوابگاه اساتيد دانشگاه دامغان
...,...,...,...,...,...,...,...,...,...,...
64,Rocha,-34.4833,-54.3333,288.16,52,0,3.87,UY,1631718466,Beleza Pura B&B
65,Belyy Yar,53.6039,91.3903,286.09,62,0,2.00,RU,1631718469,Pasportnyy Stol
66,Neyshabur,36.2133,58.7958,302.44,10,0,4.48,IR,1631718470,Amiran Hotel
67,Siteki,-26.4500,31.9500,297.05,56,0,6.45,SZ,1631718471,Siteki Hotel


In [50]:
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Morón,-34.6534,-58.6198,291.49,39,0,2.24,AR,1631718278,Hotel Morón
1,Dubai,25.2582,55.3047,309.31,56,0,3.60,AE,1631718232,"Radisson Blu Hotel, Dubai Deira Creek"
2,Brønnøysund,65.4625,12.1997,288.15,55,0,4.12,NO,1631718281,Corner Hotel
3,Saint-Pierre,-21.3393,55.4781,295.97,70,0,4.63,RE,1631718282,Lindsey Hôtel
4,Dāmghān,36.1683,54.3480,302.23,10,0,2.05,IR,1631718284,خوابگاه اساتيد دانشگاه دامغان


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [64]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))